# Trabajo Práctico NLP: Detección de Tópicos y Clasificación

**Mariela Iaccarino**

Certificación Experta en NLP - ITBA

In [1]:
!pip install datasets

In [2]:
!pip install --upgrade umap-learn

In [3]:
!pip install chromadb

In [4]:
!pip install hdbscan

In [5]:
!pip install sentence_transformers

In [6]:
!pip install BERTopic

## 1. Configuración y Carga de Datos
Importación de librerías necesarias:

In [7]:
from datasets import load_dataset
import pandas as pd
from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer
from datetime import datetime
from transformers import pipeline
import chromadb

Carga y preparación de datos:

Se carga un conjunto de datos de noticias de 5 días y se le agrega un campo de fecha.

In [8]:
import pandas as pd
from datasets import load_dataset

# Función para agregar un campo fecha a un dataset
def add_fecha_field(dataset, fecha):
    df = pd.DataFrame(dataset['train'])
    df['date'] = fecha
    return df

# Cargar un dataset (ejemplo)
ds_1 = load_dataset("jganzabalseenka/news_2024-07-01_24hs")
# Agregar el campo fecha con el valor '2024-07-01'
fecha_deseada = '2024-07-01'
df_1 = add_fecha_field(ds_1, fecha_deseada)

ds_2 = load_dataset("jganzabalseenka/news_2024-07-12_24hs")
fecha_deseada = '2024-07-12'
df_2 = add_fecha_field(ds_2, fecha_deseada)

ds_3 = load_dataset("jganzabalseenka/news_2024-07-14_24hs")
fecha_deseada = '2024-07-14'
df_3 = add_fecha_field(ds_3, fecha_deseada)

ds_4 = load_dataset("jganzabalseenka/news_2024-07-16_24hs")
fecha_deseada = '2024-07-16'
df_4 = add_fecha_field(ds_4, fecha_deseada)


ds_5 = load_dataset("jganzabalseenka/news_2024-07-19_24hs")
fecha_deseada = '2024-07-19'
df_5 = add_fecha_field(ds_5, fecha_deseada)


# Unificar los datasets en un solo DataFrame
df_list = [df_1, df_2, df_3, df_4, df_5]
df = pd.concat(df_list, ignore_index=True)

# Verificar los cambios
print(df.loc[6:16, ['title', 'date']])


                                                title        date
6   Adiós a Ismail Kadare, una luz brillante en lo...  2024-07-01
7   Cristina Kirchner: "El único que sigue creyend...  2024-07-01
8            Receta de pan de centeno, rápida y fácil  2024-07-01
9   Los nuevos eurodiputados españoles acatan hoy ...  2024-07-01
10  Portugal y Eslovenia no se sacan ventajas en l...  2024-07-01
11  Las series más esperadas que llegan a Netflix,...  2024-07-01
12  El Gobierno crea la nueva especialidad de Urge...  2024-07-01
13  Delgado, Orsi y Ojeda serían los candidatos a ...  2024-07-01
14  Nicola Peltz-Beckham ha evaluado demandar a un...  2024-07-01
15  La desoladora reflexión de Pedro Gallese tras ...  2024-07-01
16  Cuándo es firme una sentencia de divorcio cont...  2024-07-01


In [9]:
# Ver los distintos días en formato fecha
unique_dates = df['date'].unique()
print("Fechas únicas en los conjuntos de datos:")
for date in unique_dates:
    print(date)

Fechas únicas en los conjuntos de datos:
2024-07-01
2024-07-12
2024-07-14
2024-07-16
2024-07-19


In [10]:
# Dividir el dataset por días
daily_data = {date: data for date, data in df.groupby(df['date'])}

In [11]:
df.head()

,asset_id,title_ch,Asset Destination,media,impact,start_time_utc,start_time_local,entities_curated,entities,predicted_at_entities,...,entities_transformers,title,text,keywords,predicted_at_keywords,truncated_text,title_and_text,prediction_delay_predictions,prediction_delay,date
0,113231459,"Domingo, 30 de junio de 2024 (24:00 GMT)",http://infobae.com/america/agencias/2024/07/01...,Infobae,27969,2024-07-01 02:36:06,2024-06-30 23:36:06,"[Joe Biden, Consejo Nacional Electoral, CNE]","[Sergio Hernández, Marine Le Pen, Natalia Kidd...",2024-07-01 02:39:47.184910,...,"[Newsroom Infobae Nuevo, FRANCIA, París, Marin...","Domingo, 30 de junio de 2024 (24:00 GMT)","30 Jun, 2024 Por Newsroom Infobae Nuevo FRANC...","[foto, elecciones presidenciales, cumbre mundi...",2024-07-01 02:47:13.676140,"30 Jun, 2024 Por Newsroom Infobae Nuevo FRANC...","Domingo, 30 de junio de 2024 (24:00 GMT)\n30 J...",0.124025,0.185466,2024-07-01
1,113327885,"Encontraron un golpe, dos pelos y manchas roja...",http://infobae.com/sociedad/policiales/2024/07...,Infobae,181455,2024-07-01 20:49:30,2024-07-01 17:49:30,"[Carlos Pérez, Infobae]","[Mariano de Guzmán, Güemes, Alejandra Mangano,...",2024-07-01 20:49:45.393438,...,"[Loan, Federico Fahsbender Nuevo, Laudelina Pe...","Encontraron un golpe, dos pelos y manchas roja...","1 Jul, 2024 Por Federico Fahsbender Nuevo En ...","[manchas rojas, camioneta, peña, pericia, rast...",2024-07-01 20:55:24.886450,"1 Jul, 2024 Por Federico Fahsbender Nuevo En ...","Encontraron un golpe, dos pelos y manchas roja...",0.094304,0.098580,2024-07-01
2,113332680,Las cinco enfermedades por las que se ha conce...,http://infobae.com/espana/2024/07/01/las-enfer...,Infobae,9976,2024-07-01 21:41:47,2024-07-01 18:41:47,[],"[Ana, TSJA, INSS, Social, TSXG, Clínica Univer...",2024-07-01 21:43:05.015868,...,"[Diego Mariño Nuevo, España, Tribunal Superior...",Las cinco enfermedades por las que se ha conce...,"1 Jul, 2024 Por Diego Mariño Nuevo La pensión...","[incapacidad permanente, pensión vitalicia, do...",2024-07-01 21:44:05.387330,"1 Jul, 2024 Por Diego Mariño Nuevo La pensión...",Las cinco enfermedades por las que se ha conce...,0.016770,0.038441,2024-07-01
3,113222271,"Así fue el espectacular cumpleaños de Vida, la...",http://infobae.com/teleshow/2024/06/30/asi-fue...,Infobae,4465,2024-07-01 00:25:38,2024-06-30 21:25:38,"[Luisana Lopilato, Georgina Barbarossa, Daniel...","[Noah, Daniela Lopilato, Luisana, Cielo, Luisa...",2024-07-01 00:46:17.563947,...,"[Vida, Luisana Lopilato, Michael Bublé, Noah, ...","Así fue el espectacular cumpleaños de Vida, la...","30 Jun, 2024 Nuevo La creatividad fluye por la...","[cumpleaños número, sirenas, hermosa decoració...",2024-07-01 01:03:19.114070,"30 Jun, 2024 Nuevo La creatividad fluye por la...","Así fue el espectacular cumpleaños de Vida, la...",0.283764,0.628087,2024-07-01
4,113233912,Clima en Madrid: conoce el pronóstico y prepár...,http://infobae.com/espana/2024/07/01/clima-en-...,Infobae,36487,2024-07-01 03:25:55,2024-07-01 00:25:55,[],"[Agencia Estatal de Meteorología, Infobae, Aem...",2024-07-01 03:27:06.116765,...,"[Madrid, Infobae Noticias, Infobae, Agencia Es...",Clima en Madrid: conoce el pronóstico y prepár...,"1 Jul, 2024 Por Infobae Noticias Nuevo Los pr...","[clima templado, grados centígrados, inviernos...",2024-07-01 03:27:35.254670,"1 Jul, 2024 Por Infobae Noticias Nuevo Los pr...",Clima en Madrid: conoce el pronóstico y prepár...,0.008094,0.027849,2024-07-01


In [12]:
# Configurar pandas para mostrar todo el contenido de las celdas
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [13]:
# Reviso un registro de la base de datos
registro = df.iloc[[1]]


In [14]:
print(registro)

    asset_id  \
1  113327885   

                                                                                                               title_ch  \
1  Encontraron un golpe, dos pelos y manchas rojas en la camioneta del principal sospechoso por la desaparición de Loan   

                                                                                                                                                                  Asset Destination  \
1  http://infobae.com/sociedad/policiales/2024/07/01/prueba-clave-en-el-caso-loan-analizan-los-resultados-de-la-pericia-a-la-camioneta-del-principal-acusado-de-la-muerte-del-chico   

     media  impact      start_time_utc    start_time_local  \
1  Infobae  181455 2024-07-01 20:49:30 2024-07-01 17:49:30   

          entities_curated  \
1  [Carlos Pérez, Infobae]   

                                                                                                                                                                       

Stopwords en español

In [15]:
spanish_stopwords = [
    'stopwords', 'a', 'actualmente', 'adelante', 'además', 'afirmó', 'agregó',
    'ahora', 'ahí', 'al', 'algo', 'algún', 'alguna', 'algunas', 'alguno', 'algunos',
    'algún', 'alrededor', 'ambos', 'ampleamos', 'ante', 'anterior', 'antes', 'apenas',
    'aproximadamente', 'aquel', 'aquellas', 'aquellos', 'aqui', 'aquí', 'arriba', 'aseguró',
    'así', 'atras', 'aunque', 'ayer', 'añadió', 'aún', 'bajo', 'bastante', 'bien', 'buen',
    'buena', 'buenas', 'bueno', 'buenos', 'cada', 'casi', 'cerca', 'cierta', 'ciertas',
    'cierto', 'ciertos', 'cinco', 'comentó', 'como', 'con', 'conocer', 'conseguimos',
    'conseguir', 'considera', 'consideró', 'consigo', 'consigue', 'consiguen', 'consigues',
    'contra', 'cosas', 'creo', 'cual', 'cuales', 'cualquier', 'cuando', 'cuanto', 'cuatro',
    'cuenta', 'cómo', 'da', 'dado', 'dan', 'dar', 'de', 'debe', 'deben', 'debido', 'decir',
    'dejó', 'del', 'demás', 'dentro', 'desde', 'después', 'dice', 'dicen', 'dicho', 'dieron',
    'diferente', 'diferentes', 'dijeron', 'dijo', 'dio', 'donde', 'dos', 'durante', 'e',
    'ejemplo', 'el', 'ella', 'ellas', 'ello', 'ellos', 'embargo', 'empleais', 'emplean',
    'emplear', 'empleas', 'empleo', 'en', 'encima', 'encuentra', 'entonces', 'entre', 'era',
    'erais', 'eramos', 'eran', 'eras', 'eres', 'es', 'esa', 'esas', 'ese', 'eso', 'esos',
    'esta', 'estaba', 'estabais', 'estaban', 'estabas', 'estad', 'estada', 'estadas', 'estado',
    'estados', 'estais', 'estamos', 'estan', 'estando', 'estar', 'estaremos', 'estará', 'estarán',
    'estarás', 'estaré', 'estaréis', 'estaría', 'estaríais', 'estaríamos', 'estarían', 'estarías',
    'estas', 'este', 'estemos', 'esto', 'estos', 'estoy', 'estuve', 'estuviera', 'estuvierais',
    'estuvieran', 'estuvieras', 'estuvieron', 'estuviese', 'estuvieseis', 'estuviesen', 'estuvieses',
    'estuvimos', 'estuviste', 'estuvisteis', 'estuviéramos', 'estuviésemos', 'estuvo', 'está',
    'estábamos', 'estáis', 'están', 'estás', 'esté', 'estéis', 'estén', 'estés', 'ex', 'existe',
    'existen', 'explicó', 'expresó', 'fin', 'fue', 'fuera', 'fuerais', 'fueran', 'fueras', 'fueron',
    'fuese', 'fueseis', 'fuesen', 'fueses', 'fui', 'fuimos', 'fuiste', 'fuisteis', 'fuéramos',
    'fuésemos', 'gran', 'grandes', 'gueno', 'ha', 'haber', 'habida', 'habidas', 'habido', 'habidos',
    'habiendo', 'habremos', 'habrá', 'habrán', 'habrás', 'habré', 'habréis', 'habría', 'habríais',
    'habríamos', 'habrían', 'habrías', 'habéis', 'había', 'habíais', 'habíamos', 'habían', 'habías',
    'hace', 'haceis', 'hacemos', 'hacen', 'hacer', 'hacerlo', 'haces', 'hacia', 'haciendo', 'hago',
    'han', 'has', 'hasta', 'hay', 'haya', 'hayamos', 'hayan', 'hayas', 'hayáis', 'he', 'hecho',
    'hemos', 'hicieron', 'hizo', 'hoy', 'hube', 'hubiera', 'hubierais', 'hubieran', 'hubieras',
    'hubieron', 'hubiese', 'hubieseis', 'hubiesen', 'hubieses', 'hubimos', 'hubiste', 'hubisteis',
    'hubiéramos', 'hubiésemos', 'hubo', 'igual', 'incluso', 'indicó', 'informó', 'intenta', 'intentais',
    'intentamos', 'intentan', 'intentar', 'intentas', 'intento', 'ir', 'junto', 'la', 'lado', 'largo',
    'las', 'le', 'les', 'llegó', 'lleva', 'llevar', 'lo', 'los', 'luego', 'lugar', 'manera', 'manifestó',
    'mayor', 'me', 'mediante', 'mejor', 'mencionó', 'menos', 'mi', 'mientras', 'mio', 'mis', 'misma',
    'mismas', 'mismo', 'mismos', 'modo', 'momento', 'mucha', 'muchas', 'mucho', 'muchos', 'muy',
    'más', 'mí', 'mía', 'mías', 'mío', 'míos', 'nada', 'nadie', 'ni', 'ninguna', 'ningunas', 'ninguno',
    'ningunos', 'ningún', 'no', 'nos', 'nosotras', 'nosotros', 'nuestra', 'nuestras', 'nuestro',
    'nuestros', 'nueva', 'nuevas', 'nuevo', 'nuevos', 'nunca', 'o', 'ocho', 'os', 'otra', 'otras',
    'otro', 'otros', 'para', 'parece', 'parte', 'partir', 'pasada', 'pasado', 'pero', 'pesar', 'poca',
    'pocas', 'poco', 'pocos', 'podeis', 'podemos', 'poder', 'podria', 'podriais', 'podriamos', 'podrian',
    'podrias', 'podrá', 'podrán', 'podría', 'podrían', 'poner', 'por', 'por qué', 'porque', 'posible',
    'primer', 'primera', 'primero', 'primeros', 'principalmente', 'propia', 'propias', 'propio', 'propios',
    'próximo', 'próximos', 'pudo', 'pueda', 'puede', 'pueden', 'puedo', 'pues', 'que', 'quedó', 'queremos',
    'quien', 'quienes', 'quiere', 'quién', 'qué', 'realizado', 'realizar', 'realizó', 'respecto', 'sabe',
    'sabeis', 'sabemos', 'saben', 'saber', 'sabes', 'se', 'sea', 'seamos', 'sean', 'seas', 'segunda',
    'segundo', 'según', 'seis', 'ser', 'seremos', 'será', 'serán', 'serás', 'seré', 'seréis', 'sería',
    'seríais', 'seríamos', 'serían', 'serías', 'seáis', 'señaló', 'si', 'sido', 'siempre', 'siendo',
    'siete', 'sigue', 'siguiente', 'sin', 'sino', 'sobre', 'sois', 'sola', 'solamente', 'solas', 'solo',
    'solos', 'somos', 'son', 'soy', 'su', 'sus', 'suya', 'suyas', 'suyo', 'suyos', 'sí', 'sólo', 'tal',
    'también', 'tampoco', 'tan', 'tanto', 'te', 'tendremos', 'tendrá', 'tendrán', 'tendrás', 'tendré',
    'tendréis', 'tendría', 'tendríais', 'tendríamos', 'tendrían', 'tendrías', 'tened', 'teneis', 'tenemos',
    'tener', 'tenga', 'tengamos', 'tengan', 'tengas', 'tengo', 'tengáis', 'tenida', 'tenidas', 'tenido',
    'tenidos', 'teniendo', 'tenéis', 'tenía', 'teníais', 'teníamos', 'tenían', 'tenías', 'tercera', 'ti',
    'tiempo', 'tiene', 'tienen', 'tienes', 'toda', 'todas', 'todavía', 'todo', 'todos', 'total', 'trabaja',
    'trabajais', 'trabajamos', 'trabajan', 'trabajar', 'trabajas', 'trabajo', 'tras', 'trata', 'través',
    'tres', 'tu', 'tus', 'tuve', 'tuviera', 'tuvierais', 'tuvieran', 'tuvieras', 'tuvieron', 'tuviese',
    'tuvieseis', 'tuviesen', 'tuvieses', 'tuvimos', 'tuviste', 'tuvisteis', 'tuviéramos', 'tuviésemos',
    'tuvo', 'tuya', 'tuyas', 'tuyo', 'tuyos', 'tú', 'ultimo', 'un', 'una', 'unas', 'uno', 'unos', 'usa',
    'usais', 'usamos', 'usan', 'usar', 'usas', 'uso', 'usted', 'va', 'vais', 'valor', 'vamos', 'van',
    'varias', 'varios', 'vaya', 'veces', 'ver', 'verdad', 'verdadera', 'verdadero', 'vez', 'vosotras',
    'vosotros', 'voy', 'vuestra', 'vuestras', 'vuestro', 'vuestros', 'y', 'ya', 'yo', 'él', 'éramos',
    'ésta', 'éstas', 'éste', 'éstos', 'última', 'últimas', 'último', 'últimos'
]


Configuración del Modelo de Embeddings y Otros Modelos:

In [16]:
# Configuración del modelo de embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Configuración de UMAP para reducción de dimensionalidad
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')

# Configuración de HDBSCAN para clustering
hdbscan_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)


# Configuración de ChromaDB para almacenar los embeddings
client = chromadb.Client()
collection_name = "news_topics"
if collection_name in [coll.name for coll in client.list_collections()]:
    collection = client.get_collection(collection_name)
else:
    collection = client.create_collection(collection_name)

# Configuración de pipelines para NER y análisis de sentimiento
ner_pipeline = pipeline('ner', model='dbmdz/bert-large-cased-finetuned-conll03-english')
sentiment_pipeline = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

## 2. Procesamiento y Detección de Tópicos Diarios

**Análisis de los Tópicos Generados (Ejemplo con la base ds_1 del 2024-07-01):**

Para analizar los tópicos generados en el dataset del 2024-07-01, se podría utilizar el siguiente bloque de código:

In [17]:
# Obtener los textos del dataset específico
texts = df[df['date'] == '2024-07-01']['title_ch'].tolist()

# Definir el vectorizador usando todas las entidades y keywords únicas del dataset
entities = set(sum(list([list(e) for e in df_1['entities_transformers'].values]), []))
keywords = set(sum(list([list(e) for e in df_1['keywords'].values]), []))
all_tokens = list(entities.union(keywords))

# Configurar el vectorizador
tf_vectorizer = CountVectorizer(
    ngram_range=(1, 3),
    stop_words=spanish_stopwords,
    lowercase=False,
    vocabulary=all_tokens,
)

topic_model_1 = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=tf_vectorizer,
    ctfidf_model=ClassTfidfTransformer(),
    representation_model=KeyBERTInspired(),
    language='spanish'
)
topics_1, probs_1 = topic_model_1.fit_transform(texts)

# Mostrar los tópicos generados
for topic in set(topics_1):
    topic_keywords = topic_model_1.get_topic(topic)
    print(f"Tópico {topic}: {', '.join([word[0] for word in topic_keywords])}")


/usr/local/lib/python3.10/dist-packages/bertopic/vectorizers/_ctfidf.py:82: RuntimeWarning: divide by zero encountered in divide
  idf = np.log((avg_nr_samples / df) + 1)


Tópico 0: Gran Hermano, Gran hermano, eliminación, reacción, hermano, tremenda reacción, Gran, El Chino, emotivo reencuentro, hermana
Tópico 1: fútbol femenino, Fútbol Femenino, fútbol mundial, Liga Nacional, Deportivo, fútbol masculino, Deportivo Maipú, Primera Nacional, Atlético Rafaela, clubes
Tópico 2: combustibles, aumento combustibles, combustibles nafta, aumento combustible, combustible, petroleras, petroleros, temprana eliminación, Löffler, potencial turístico
Tópico 3: primeras imágenes, mejores fotos, Nacho Castañares, El Conejo, tierna foto, foto íntima, unidad, foto, relación, fotos
Tópico 4: El Turco, feriado nacional, Fundación Sendero, Turco Julio, puente, El Quillá, El Oso, feriado, viento fuerte, posibles delanteros
Tópico 5: pesos, millones pesos, peso, Barrio Obrero, cotización, plazo fijo, euros, kilos, costo, Diario La
Tópico 6: inmunidad presidencial, inmunidad, realidad, inmunidad parcial, elecciones, conversaciones serias, carrera presidencial, inmunidad absolut

In [18]:
df_1['topic'] = topics_1
df_1['probs'] = probs_1

In [19]:
df_1[df_1['topic']==1][['title_ch', 'topic', 'probs']]

,title_ch,topic,probs
37,"Por huracán Beryl, la Federación Colombiana de Fútbol canceló importante evento de la selección Colombia",1,1.000000
251,"El fútbol mundial se rinde a Nico Williams y Lamine Yamal: goles, bailes y un `piedra, papel o tijera` de un dúo mágico",1,0.556761
344,La segunda oportunidad de Koeman,1,0.614043
381,"Fuzato, Carmona, Latasa, Óscar Rodríguez, Greenwood e Ilaix finalizan sus cesiones",1,0.478113
590,Forum Jet es el campeón de la Liga Monumental venezolana,1,0.787829
599,No está `fuera de tu liga`: la gente tiende a casarse con personas tan atractivas como ellas,1,0.749822
645,Mundial U17 de Básquetbol: derrota de Argentina,1,1.000000
781,Liga Senior de Fútbol de Olavarría: Continúan los Playoff,1,1.000000
824,Liga Sanlorencina: San Martín le ganó el clásico a PSM y se quedó con la punta de la tabla,1,0.468020
872,"En su segunda semana, los Juegos Intercolegiales Deportivos llegan a Villa Mercedes y al Valle del Conlara",1,0.827427


In [20]:
len(topics_1), len(df_1)

(16025, 16025)

In [21]:
topic_model_1.topic_representations_[1]

[('fútbol femenino', 0.6532519),
 ('Fútbol Femenino', 0.65325177),
 ('fútbol mundial', 0.6287671),
 ('Liga Nacional', 0.62420774),
 ('Deportivo', 0.6125127),
 ('fútbol masculino', 0.61236846),
 ('Deportivo Maipú', 0.60699874),
 ('Primera Nacional', 0.5912842),
 ('Atlético Rafaela', 0.571941),
 ('clubes', 0.5575856)]

In [22]:
#topic_model_1.topic_representations_

In [23]:
topic_model_1.topic_embeddings_.shape

(255, 384)

 Exploratorio Extracción Topicos Base 1

In [24]:
topic_model_1.get_topics()

{-1: [('Selección Argentina', 0.5552928),
  ('Ecuador', 0.53424466),
  ('México', 0.51062816),
  ('Por', 0.49758047),
  ('Boca', 0.49218458),
  ('Argentina', 0.49139914),
  ('argentina', 0.49139914),
  ('Es', 0.4613794),
  ('frente', 0.45617777),
  ('Mercado', 0.43029672)],
 0: [('Gran Hermano', 0.64560866),
  ('Gran hermano', 0.64560866),
  ('eliminación', 0.5591891),
  ('reacción', 0.5250456),
  ('hermano', 0.5219888),
  ('tremenda reacción', 0.5110121),
  ('Gran', 0.47084236),
  ('El Chino', 0.45584652),
  ('emotivo reencuentro', 0.45564103),
  ('hermana', 0.45343828)],
 1: [('fútbol femenino', 0.6532519),
  ('Fútbol Femenino', 0.65325177),
  ('fútbol mundial', 0.6287671),
  ('Liga Nacional', 0.62420774),
  ('Deportivo', 0.6125127),
  ('fútbol masculino', 0.61236846),
  ('Deportivo Maipú', 0.60699874),
  ('Primera Nacional', 0.5912842),
  ('Atlético Rafaela', 0.571941),
  ('clubes', 0.5575856)],
 2: [('combustibles', 0.7115105),
  ('aumento combustibles', 0.6849488),
  ('combustible

In [25]:
topic_model_1.visualize_topics()

In [26]:
#topic_model_1.visualize_hierarchy()

In [27]:
topic_model_1.visualize_term_rank()


**Definición de Funciones para el Procesamiento de todos los datasets:**

La función **get_closest_topics** tiene como objetivo encontrar los tópicos más cercanos a un nuevo documento basado en su embedding. Utiliza una base de datos vectorial para realizar esta búsqueda, devolviendo los tópicos más similares según una métrica de distancia (coseno en este caso).

In [28]:
def get_closest_topics(embedding, date, top_k=1):
    results = collection.query(query_embeddings=[embedding], n_results=top_k)
    return results


embedding (list or array): Este parámetro representa el embedding del documento nuevo, es decir, una representación vectorial del documento en el espacio de embeddings. Los embeddings son obtenidos utilizando un modelo de transformación de frases (en este caso, SentenceTransformer).


*   collection.query: Esta llamada realiza una búsqueda en la base de datos vectorial.
*   query_embeddings: Especifica los embeddings con los que queremos comparar los tópicos almacenados en la base de datos. En este caso, estamos buscando tópicos similares al embedding del documento nuevo.
*   n_results: Especifica cuántos resultados (tópicos) queremos obtener. Aquí estamos buscando los top_k tópicos más cercanos.

La base de datos vectorial almacena los embeddings de los tópicos previamente calculados. Al realizar esta consulta, se utiliza una métrica de distancia (como la similitud de coseno) para encontrar los embeddings más cercanos al embedding proporcionado.

La función **generate_topic_name** tiene como objetivo crear un nombre para un tópico (tema) basado en una lista de palabras clave (keywords) asociadas a ese tópico

In [29]:
def generate_topic_name(keywords):
    # Crear un nombre de tópico concatenando las primeras 4 palabras clave
    return " ".join(keywords[:4])


**Función para procesar documentos diarios**

La función **process_daily_documents** está diseñada para procesar documentos de noticias recibidos diariamente, detectar y agrupar tópicos usando BERTopic, y almacenar estos tópicos en una base de datos vectorial.

In [30]:

# Definir el vectorizador usando todas las entidades y keywords únicas del dataset
entities = set(sum(list([list(e) for e in df['entities_transformers'].values]), []))
keywords = set(sum(list([list(e) for e in df['keywords'].values]), []))
all_tokens = list(entities.union(keywords))

# Configurar el vectorizador
tf_vectorizer = CountVectorizer(
    ngram_range=(1, 3),
    stop_words=spanish_stopwords,
    lowercase=False,
    vocabulary=all_tokens,
)

In [31]:

topic_id_counter = 1

def process_daily_documents(daily_data, collection):
    global topic_id_counter
    daily_summary = {}  # Diccionario para el resumen diario de tópicos
    for date, data in daily_data.items():
        texts = data['title_ch'].tolist()
        if len(texts) < 15:
            continue

        # Filtrar textos vacíos
        texts = [text for text in texts if text.strip()]
        if not texts:
            continue

        topic_model = BERTopic(
            embedding_model=embedding_model,
            umap_model=umap_model,
            hdbscan_model=hdbscan_model,
            vectorizer_model=tf_vectorizer,
            ctfidf_model=ClassTfidfTransformer(),
            representation_model=KeyBERTInspired(),
            language='spanish'
        )

        topics, probs = topic_model.fit_transform(texts)
        new_topics_today = 0  # Contador de nuevos tópicos para el día
        existing_topics_today = 0  # Contador de tópicos existentes para el día
        topic_documents_count = {}  # Diccionario para contar documentos por tópico

        for text, topic, prob in zip(texts, topics, probs):
            embedding = embedding_model.encode([text]).tolist()
            closest_topics = get_closest_topics(embedding[0], date)

            if closest_topics and 'documents' in closest_topics and closest_topics['documents']:
                valid_distances = [(doc, dist[0]) for doc, dist in zip(closest_topics['documents'], closest_topics['distances']) if dist]
                if valid_distances:
                    closest_topic = min(valid_distances, key=lambda x: x[1])
                    if closest_topic[1] < 0.2:
                        topic_id = str(closest_topic[0])  # Convertir a cadena de texto
                        existing_topics_today += 1
                    else:
                        keywords = [word[0] for word in topic_model.get_topic(topic)[:4]]  # Tomar las 4 principales keywords
                        topic_name = generate_topic_name(keywords)
                        topic_id = str(topic_id_counter)
                        topic_id_counter += 1
                        new_topics_today += 1
                        collection.add(documents=[text], metadatas=[{
                            'date': str(date),
                            'topic_id': topic_id,
                            'keywords': ", ".join(keywords),  # Convertir la lista de keywords a una cadena de texto
                            'threshold': 0.2,  # Umbral de detección
                            'topic_name': topic_name
                        }], ids=[topic_id], embeddings=embedding)
                else:
                    keywords = [word[0] for word in topic_model.get_topic(topic)[:4]]  # Tomar las 4 principales keywords
                    topic_name = generate_topic_name(keywords)
                    topic_id = str(topic_id_counter)
                    topic_id_counter += 1
                    new_topics_today += 1
                    collection.add(documents=[text], metadatas=[{
                        'date': str(date),
                        'topic_id': topic_id,
                        'keywords': ", ".join(keywords),  # Convertir la lista de keywords a una cadena de texto
                        'threshold': 0.2,  # Umbral de detección
                        'topic_name': topic_name
                    }], ids=[topic_id], embeddings=embedding)
            else:
                keywords = [word[0] for word in topic_model.get_topic(topic)[:4]]  # Tomar las 4 principales keywords
                topic_name = generate_topic_name(keywords)
                topic_id = str(topic_id_counter)
                topic_id_counter += 1
                new_topics_today += 1
                collection.add(documents=[text], metadatas=[{
                    'date': str(date),
                    'topic_id': topic_id,
                    'keywords': ", ".join(keywords),  # Convertir la lista de keywords a una cadena de texto
                    'threshold': 0.2,  # Umbral de detección
                    'topic_name': topic_name
                }], ids=[topic_id], embeddings=embedding)

            if topic_id not in topic_documents_count:
                topic_documents_count[topic_id] = 0
            topic_documents_count[topic_id] += 1

        daily_summary[date] = {
            'total_documents': len(texts),
            'total_topics': len(topic_documents_count),
            'new_topics': new_topics_today,
            'existing_topics': existing_topics_today
        }

    return daily_summary


In [32]:

# Procesar documentos diarios
daily_summary = process_daily_documents(daily_data, collection)
print("Resumen diario de tópicos:", daily_summary)

/usr/local/lib/python3.10/dist-packages/bertopic/vectorizers/_ctfidf.py:82: RuntimeWarning:

divide by zero encountered in divide

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning:

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.

/usr/local/lib/python3.10/dist-packages/bertopic/vectorizers/_ctfidf.py:82: RuntimeWarning:

divide by zero encountered in divide

/usr/local/lib/python3.10/dist-packages/bertopic/vectorizers/_ctfidf.py:82: RuntimeWarning:

divide by zero encountered in divide

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning:

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.

/usr/local/lib/python3.10/dist-packages/bertopic/vectorizers/_ctfidf.py:82: RuntimeWarning:

divide by zero encountered

Resumen diario de tópicos: {'2024-07-01': {'total_documents': 16025, 'total_topics': 14044, 'new_topics': 12453, 'existing_topics': 3572}, '2024-07-12': {'total_documents': 17843, 'total_topics': 15624, 'new_topics': 13733, 'existing_topics': 4110}, '2024-07-14': {'total_documents': 7185, 'total_topics': 6582, 'new_topics': 5787, 'existing_topics': 1398}, '2024-07-16': {'total_documents': 17283, 'total_topics': 14957, 'new_topics': 12891, 'existing_topics': 4392}, '2024-07-19': {'total_documents': 17562, 'total_topics': 15426, 'new_topics': 13386, 'existing_topics': 4176}}


Este bloque de código realiza las siguientes acciones:

1) Aplica el Modelo BERTopic:
Ajusta el modelo BERTopic a los textos diarios y obtiene los tópicos y probabilidades para cada texto.

2) Itera sobre Textos y Tópicos:
Para cada texto, tópico y probabilidad, genera un embedding del texto.

3) Asigna Identificadores Únicos:
Asigna un identificador único a cada nuevo tópico.

4) Almacena en la Base de Datos Vectorial:
Almacena el texto, su metadata y su embedding en la base de datos vectorial para futuras consultas y comparaciones.

Este proceso permite la detección, agrupación y almacenamiento de tópicos de manera eficiente, facilitando la clasificación y análisis de nuevos documentos a medida que se reciben.

## 3. Procesamiento de Documentos Nuevos


Función process_new_document
Esta función se encargará de generar el embedding del nuevo documento, compararlo con los embeddings de tópicos almacenados, y extraer entidades y análisis de sentimiento.

In [34]:
def process_new_document(title, text, date):
    embedding = embedding_model.encode([text]).tolist()              #Genera una representación vectorial (embedding) del nuevo documento.
    closest_topics = get_closest_topics(embedding[0], date)          #Obtener los Tópicos Más Cercanos.

    #get_closest_topics:Es una función que consulta la base de datos vectorial para encontrar los tópicos más cercanos al embedding proporcionado. Esta función retorna los tópicos y sus distancias en orden de proximidad.

    # Verificar la Existencia de Tópicos Cercanos Válidos
    if closest_topics and 'documents' in closest_topics and closest_topics['documents']:
        valid_distances = [(doc, dist[0]) for doc, dist in zip(closest_topics['documents'], closest_topics['distances']) if dist] # Filtrar las Distancias Válidas
        if valid_distances:
            closest_topic = min(valid_distances, key=lambda x: x[1])   #Seleccionar el Tópico Más Cercano
            topic_id = closest_topic[0]                                #Determinar el ID del Tópico Más Cercano
        else:
            topic_id = None
    else:
        topic_id = None

    entities = ner_pipeline(text)                          #Extracción de Entidades y Análisis de Sentimiento:
    sentiment = sentiment_pipeline(text)

    return {
        'topic_id': topic_id,
        'entities': entities,
        'sentiment': sentiment
    }


**Resumen**

process_daily_documents: Aplica BERTopic para detectar tópicos en los textos diarios y almacena estos tópicos junto con sus embeddings en la base de datos vectorial.

process_new_document: Compara el nuevo documento con los tópicos almacenados, asigna el documento al tópico más cercano o marca como nuevo, y extrae entidades y análisis de sentimiento.

Con estas funciones, el sistema es capaz de detectar, agrupar y almacenar tópicos de manera eficiente, así como clasificar y analizar nuevos documentos a medida que se reciben.

## 4. Ejecución del Proceso

In [35]:
# Procesar documentos diarios
process_daily_documents(daily_data, collection)

# Ejemplo de un nuevo documento
new_doc_title = "Bauti Ganador de GH"
new_doc_text = "Bautista,  fue el ganador de la edición de GH de 2024"
new_doc_date = pd.to_datetime("2024-07-22")

# Procesar el nuevo documento para encontrar su tópico más cercano y extraer entidades y sentimiento
result = process_new_document(new_doc_title, new_doc_text, new_doc_date)

# Mostrar los resultados de la inferencia
print(result)

/usr/local/lib/python3.10/dist-packages/bertopic/vectorizers/_ctfidf.py:82: RuntimeWarning:

divide by zero encountered in divide

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning:

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.

/usr/local/lib/python3.10/dist-packages/bertopic/vectorizers/_ctfidf.py:82: RuntimeWarning:

divide by zero encountered in divide

/usr/local/lib/python3.10/dist-packages/bertopic/vectorizers/_ctfidf.py:82: RuntimeWarning:

divide by zero encountered in divide

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning:

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.

/usr/local/lib/python3.10/dist-packages/bertopic/vectorizers/_ctfidf.py:82: RuntimeWarning:

divide by zero encountered

{'topic_id': ['Denisse González contó por qué no está enamorada de Bautista Mascia de Gran Hermano 2023'], 'entities': [{'entity': 'I-PER', 'score': 0.9701536, 'index': 1, 'word': 'Ba', 'start': 0, 'end': 2}, {'entity': 'I-PER', 'score': 0.68906677, 'index': 2, 'word': '##uti', 'start': 2, 'end': 5}, {'entity': 'I-PER', 'score': 0.82399064, 'index': 3, 'word': '##sta', 'start': 5, 'end': 8}, {'entity': 'I-ORG', 'score': 0.8116116, 'index': 17, 'word': 'G', 'start': 43, 'end': 44}], 'sentiment': [{'label': 'POSITIVE', 'score': 0.9079534411430359}]}


In [36]:
# Ejemplo de un nuevo documento
new_doc_title = "Argentina campeon"
new_doc_text = "Finalizo la copa america y Argentina salio campeon"
new_doc_date = pd.to_datetime("2024-07-22")

# Procesar el nuevo documento para encontrar su tópico más cercano y extraer entidades y sentimiento
result = process_new_document(new_doc_title, new_doc_text, new_doc_date)

# Mostrar los resultados de la inferencia
print(result)

{'topic_id': ['Argentina bicampeón de la Copa América'], 'entities': [{'entity': 'I-LOC', 'score': 0.9896319, 'index': 11, 'word': 'Argentina', 'start': 27, 'end': 36}], 'sentiment': [{'label': 'POSITIVE', 'score': 0.9376359581947327}]}
